# 5장, 판다스 시작하기

https://wesmckinney.com/book/pandas-basics


In [1]:
!pip install -q numpy pandas
# !pip install -Uqq datasets # in HuggingFace

In [1]:
import numpy as np
import pandas as pd
# import torch

from pandas import Series, DataFrame

## 5.1 pandas 데이터 구조 소개
### 시리즈


### 데이터 프레임

In [3]:
data = {"state": ["Ohio", "Ohio", "Ohio", "Nevada", "Nevada", "Nevada"],
        "year": [2000, 2001, 2002, 2001, 2002, 2003],
        "pop": [1.5, 1.7, 3.6, 2.4, 2.9, 3.2]}
frame = pd.DataFrame(data)
frame

,state,year,pop
0,Ohio,2000,1.5
1,Ohio,2001,1.7
2,Ohio,2002,3.6
3,Nevada,2001,2.4
4,Nevada,2002,2.9
5,Nevada,2003,3.2


In [4]:
frame.head(4) # frame.tail()

,state,year,pop
0,Ohio,2000,1.5
1,Ohio,2001,1.7
2,Ohio,2002,3.6
3,Nevada,2001,2.4


In [5]:
pd.DataFrame(data, columns=["year", "state", "pop"]) # 컬럼의 순서를 커스터마이즈..

,year,state,pop
0,2000,Ohio,1.5
1,2001,Ohio,1.7
2,2002,Ohio,3.6
3,2001,Nevada,2.4
4,2002,Nevada,2.9
5,2003,Nevada,3.2


In [6]:
# 열 추출
frame['state']  # == frame.state

0      Ohio
1      Ohio
2      Ohio
3    Nevada
4    Nevada
5    Nevada
Name: state, dtype: object

In [7]:
#frame[2]
# 이렇게는 사용 할 수 없음.

### 인덱스 객체

https://wesmckinney.com/book/pandas-basics#pandas_index_objects

pandas의 Index 객체는 축 레이블(DataFrame의 열 이름 포함)과 기타 메타데이터(축 이름 등)를 저장합니다. 

Series 또는 DataFrame을 생성할 때 사용하는 모든 배열이나 기타 레이블 시퀀스는 내부적으로 Index로 변환됩니다.


In [2]:
obj = pd.Series(np.arange(3), index=["a", "b", "c"])

index = obj.index
print(index)
# Index(['a', 'b', 'c'], dtype='object')

print(index[1:])
# Index(['b', 'c'], dtype='object')

Index(['a', 'b', 'c'], dtype='object')
Index(['b', 'c'], dtype='object')


In [ ]:
# 인덱스 객체는 변경할 수 없으므로 사용자가 수정할 수 없습니다.

try:
    index[1] = "d"  # TypeError
except Exception as e:
    print(e.__class__, e)


<class 'TypeError'> Index does not support mutable operations


In [ ]:
# 불변성 덕분에 데이터 구조 간에 Index 객체를 공유하는 것이 더 안전해집니다.

labels = pd.Index(np.arange(3))
print(labels)
# Index([0, 1, 2], dtype='int64')

# 불변성 덕분에, 이렇게 index 로 지정한 객체는 참조가 서로 '공유' 된다.
obj2 = pd.Series([1.5, -2.5, 0], index=labels)
print(obj2)
# 0    1.5
# 1   -2.5
# 2    0.0
# dtype: float64

# 지정한 원래 그 객체가 맞음.
print(obj2.index is labels)
# True

Index([0, 1, 2], dtype='int64')
0    1.5
1   -2.5
2    0.0
dtype: float64
True


In [ ]:
# 인덱스는 배열과 유사할 뿐만 아니라 고정 크기 세트처럼 동작합니다.
populations = {"Ohio": {2000: 1.5, 2001: 1.7, 2002: 3.6},
              "Nevada": {2001: 2.4, 2002: 2.9}}
frame3 = pd.DataFrame(populations)
print(frame3)
# state  Ohio  Nevada
# year
# 2000    1.5     NaN
# 2001    1.7     2.4
# 2002    3.6     2.9

print(frame3.columns)
# Index(['Ohio', 'Nevada'], dtype='object')

print("Ohio" in frame3.columns)
# True

print(2003 in frame3.index)
# False

      Ohio  Nevada
2000   1.5     NaN
2001   1.7     2.4
2002   3.6     2.9
Index(['Ohio', 'Nevada'], dtype='object')
True
False


In [8]:
# 중복 레이블을 포함할 수 있지만..

pd.Index(["foo", "foo", "bar", "bar"])
# Index(['foo', 'foo', 'bar', 'bar'], dtype='object')


# set()와 유사한 여러 메소드를 제공한다.
# union(), intersection(), ...

Index(['foo', 'foo', 'bar', 'bar'], dtype='object')


## 5.2 필수 기능

### 재색인화
### 축에서 항목 삭제


### 인덱싱, 선택 및 필터링

#### Series 인덱싱

In [8]:
obj1 = pd.Series([1, 2, 3], index=[2, 0, 1])
obj2 = pd.Series([1, 2, 3], index=["a", "b", "c"])
obj1,obj2

(2    1
 0    2
 1    3
 dtype: int64,
 a    1
 b    2
 c    3
 dtype: int64)

In [9]:
obj2['a':'b']

a    1
b    2
dtype: int64

#### DataFrame 인덱싱

기본적으로는 컬럼 인덱싱이다.

In [10]:
data = pd.DataFrame(np.arange(16).reshape((4, 4)),
   index=["Ohio", "Colorado", "Utah", "New York"],
   columns=["one", "two", "three", "four"])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [11]:
# 두 개의 axis 가 있다고 보면, 한 axis 를 인덱싱 하는 것인데 기본적으로 컬럼 인덱싱을 한다는 것이 좀 의외이긴 하다.

data['two']

Ohio         1
Colorado     5
Utah         9
New York    13
Name: two, dtype: int64

In [12]:
data[['two', 'three']]

,two,three
Ohio,1,2
Colorado,5,6
Utah,9,10
New York,13,14


In [13]:
# 이건 일반적이지 않은 특수 사용 예시이다. 너무 확장 해석을 하지는 말자.
# 1. 슬라이싱으로 column 이 아닌 row 를 추출한다.
#
data[:2]

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7


In [14]:
# bool 수식으로 인덱싱 하는 것도 특수 사용 예시.
data[ data['three'] > 7 ]

,one,two,three,four
Utah,8,9,10,11
New York,12,13,14,15


In [15]:
data < 5

,one,two,three,four
Ohio,True,True,True,True
Colorado,True,False,False,False
Utah,False,False,False,False
New York,False,False,False,False


In [16]:
# loc 을 사용해야 제대로 된 2차 배열 처럼 간주된다.
#
data.loc["Colorado"]
#
# 결과는 Series 인데 세로로 표시되어서 좀 이상하게 보이긴 함.

one      4
two      5
three    6
four     7
Name: Colorado, dtype: int64

In [17]:
data.loc[["Colorado", "New York"]]

,one,two,three,four
Colorado,4,5,6,7
New York,12,13,14,15


In [18]:
# 행, 열을 모두 인덱싱. 콤마를 추가하여 두 개의 인덱스를 제공.

data.loc["Colorado", "two"]

np.int64(5)

In [19]:
# 둘 중 리스트(시퀀스) 가 포함되면 차원이 줄어들 지 않음.
data.loc["Colorado", ["two", "three"]]

two      5
three    6
Name: Colorado, dtype: int64

In [20]:
# 값은 하나 뿐이지만, 차원을 줄어들지 않아서 여전히 DataFrame 이다.
data.loc[["Colorado"], ["two"]]

,two
Colorado,5


In [21]:
# 여기에서 슬라이스 사용 가능. 반드시 loc, iloc 에서만 쓸 수 있다.
data.loc[:'Colorado', 'two':]

,two,three,four
Ohio,1,2,3
Colorado,5,6,7


In [22]:
# 이런 직접 access 는 사용하지 않는 것이 좋다.
try:
    data['Colorado']
except Exception as e:
    print(e.__class__, e)

<class 'KeyError'> 'Colorado'


- 결론
  - 항상 `.loc[]`, `.iloc[]` 을 활용하자!


#### 체인 인덱싱 문제

In [23]:
print(data)

# 이런 식의 체인 인덱싱은 좋지 않다!  경고가 발생해서 문제를 알려주긴 하지만...
data.loc[data.three == 6]["three"] = -1
data

          one  two  three  four
Ohio        0    1      2     3
Colorado    4    5      6     7
Utah        8    9     10    11
New York   12   13     14    15


/var/folders/gk/5yll9c4s4574ff6wr892k56r0000gn/T/ipykernel_49835/2318553023.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[data.three == 6]["three"] = -1


,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [24]:
# 한번의 인덱싱으로 끝내야 한다!
#
data.loc[data.three == 6, "three"] = -1
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,-1,7
Utah,8,9,10,11
New York,12,13,14,15


### 산술 및 데이터 정렬

### 함수 적용 및 매핑

In [25]:
frame = pd.DataFrame(np.random.standard_normal((4, 3)),
                     columns=list("bde"),
                     index=["Utah", "Ohio", "Texas", "Oregon"])
frame

,b,d,e
Utah,-1.273625,0.588638,-0.761343
Ohio,-0.479537,-1.583048,-0.249228
Texas,0.010981,0.619745,0.010578
Oregon,0.416891,0.943104,0.593449


In [26]:
# NumPy ufunc(요소별 배열 메서드)도 pandas 객체와 함께 작동가능.
np.abs(frame)

,b,d,e
Utah,1.273625,0.588638,0.761343
Ohio,0.479537,1.583048,0.249228
Texas,0.010981,0.619745,0.010578
Oregon,0.416891,0.943104,0.593449


In [27]:
# 1차원 배열의 각 열이나 행에 함수를 적용
# DataFrame.apply

# f1 의 인자 x 는 요소 하나가 전달되는게 아니다! 하나의 행 또는 하나의 열이 전달된다. (아마도 Series 객체로)
# f1 에서 리턴하는 값이 scalar 이면 원본 df 에서 차원이 축소 된 df 가 apply()에서 리턴될 것이다.
def f1(x):
    return x.max() - x.min() # 이 f1 은 scalar 를 리턴

frame.apply(f1)
# 리턴 값은 아마도 frame 보다 차원이 줄어든 Series 일 것임.

# 열에 적용하는 것인지, 행에 적용하는 것인지, 어떻게 구분하는가?? 그것은 axis 선택 인자로 판단한다.
# 결과로 리턴되는 Series 의 인덱스가, [b, d, e] 이므로 첫번째 axis가 제거되었음.

b    1.690517
d    2.526152
e    1.354793
dtype: float64

In [28]:
# 보통 자주 쓰이는 패턴은 다음과 같이 특정 컬럼(feature)에 대해서 매핑 적용하는 것.
#    df['column'] = df['column'].apply(f1)

# 가장 일반적인 배열 통계(예: sum, mean) 중 다수는 DataFrame 메서드로 이미 제공한다.
# 따라서 이런 용도로는 apply 를 쓸 필요가 없다.

In [29]:
frame.apply(f1, axis='columns')
# 또는 frame.apply(f1, axis=1)

Utah      1.862263
Ohio      1.333821
Texas     0.609166
Oregon    0.526213
dtype: float64

In [30]:
# test
np.sqrt([4, 9])

array([2., 3.])

In [31]:
# 이건 또 새로운 발상. 아예 인자 x 와는 전혀 다른 새로운 Series 를 리턴할 수도 있다.
def f2(x):
    return pd.Series([x.min(), x.max()], index=["min", "max"])

print(frame)
print(frame.apply(f2))
print(frame.apply(f2, axis=1))

               b         d         e
Utah   -1.273625  0.588638 -0.761343
Ohio   -0.479537 -1.583048 -0.249228
Texas   0.010981  0.619745  0.010578
Oregon  0.416891  0.943104  0.593449
            b         d         e
min -1.273625 -1.583048 -0.761343
max  0.416891  0.943104  0.593449
             min       max
Utah   -1.273625  0.588638
Ohio   -1.583048 -0.249228
Texas   0.010578  0.619745
Oregon  0.416891  0.943104


In [32]:
# 요소 별 매핑은 apply() 가 아니고 applymap() 이다.
#
# frame 의 각 부동 소수점 값에서 형식이 지정된 문자열을 계산하고 싶다고 가정해 보겠습니다. applymap() 으로 다음과 같이 할 수 있습니다.
def my_format(x):
    return f"{x:.2f}"

print(frame.applymap(my_format))
# FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.

print(frame.map(my_format))


            b      d      e
Utah    -1.27   0.59  -0.76
Ohio    -0.48  -1.58  -0.25
Texas    0.01   0.62   0.01
Oregon   0.42   0.94   0.59
            b      d      e
Utah    -1.27   0.59  -0.76
Ohio    -0.48  -1.58  -0.25
Texas    0.01   0.62   0.01
Oregon   0.42   0.94   0.59


/var/folders/gk/5yll9c4s4574ff6wr892k56r0000gn/T/ipykernel_49835/3515542045.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  print(frame.applymap(my_format))


In [33]:
# .applymap() 이었던 이유는, Series 에 .map() 이라는 메소드가 있기 때문이라는데,
# 그게 왜 DataFrame.map() 라는 이름을 쓰지 못하는 이유가 될 수 있는지,
# 그리고, 그렇다면 지금은 왜 DataFrame.map() 을 사용할 수 있는지.. 는 알 수 없다.

frame["e"].map(my_format)

Utah      -0.76
Ohio      -0.25
Texas      0.01
Oregon     0.59
Name: e, dtype: object

### 정렬 및 순위
### 중복 레이블이 있는 축 인덱스

## 5.3 기술 통계 요약 및 계산
